In [1]:
# Useful starting lines
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from preprocessing_helpers import *
from Basics import *
#%load_ext autoreload
#%autoreload 2

In [2]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
# split test-train set
indx = np.arange(y.shape[0])
np.random.shuffle( indx )
ind_max = int(y.shape[0] * (0.8))

trainY , trainF = y[indx[:ind_max]] , tX[indx[:ind_max]]
testY , testF = y[indx[ind_max:]] , tX[indx[ind_max:]]

In [4]:
####### Preprocessing ######################################################################

In [5]:
# normalize
trainF , mean , std = standardize(trainF)
trainF = np.insert(trainF ,0 , np.ones(trainF.shape[0]),axis = 1) # add bias
testF = (testF - mean)/std 
testF = np.insert(testF ,0 , np.ones(testF.shape[0]),axis = 1) # add bias

In [6]:
# replace -999

In [7]:
# replace outliers

In [8]:
# dataset balencing

In [9]:
# feature selection expansion
trainF_sel = select_and_expand_f(trainF)
testF_sel = select_and_expand_f(testF)

In [10]:
###### Training/Testing ######################################################################

In [11]:
w_initial = np.random.rand(trainF_sel.shape[1])

w,_ = least_squares_GD( trainY,trainF_sel ,w_initial,200,0.1)

get_accuracy(w , testY ,testF_sel )

0.78092

In [ ]:
w,_ = logistic_regression( trainY,trainF_sel ,w_initial,200,0.1 )
get_accuracy_logistic(w , testY ,testF_sel )